# 🔍 Comprehensive Model & Strategy Comparison Analysis

This notebook provides a comprehensive analysis comparing:
- **Inter-model performance**: Different LLMs (qwen2.5_14b, deepseek-coder_33b, llama3.1_8b, etc.)
- **Strategy effectiveness**: Different prompt strategies (base_version, with_geom, with_geom_time)

## 📁 Data Structure
- `results/middle/`: Contains prediction results from all models
- `results/penultimate/`: Additional results dataset

Each model has three strategy subdirectories:
1. **base_version**: Basic tourist mobility prediction
2. **with_geom**: Enhanced with geospatial features
3. **with_geom_time**: Enhanced with both geospatial and temporal features

## 📊 Outputs
- Interactive visualizations for model and strategy comparison
- Canva-ready CSV exports for presentation graphics
- Statistical analysis and performance insights

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import ast
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 11

# Create output directories
output_dir = Path('../csv_for_canva_comprehensive')
output_dir.mkdir(exist_ok=True)

plots_dir = Path('../plots_comprehensive')
plots_dir.mkdir(exist_ok=True)

print(f"✅ Setup complete")
print(f"📁 CSV output directory: {output_dir.absolute()}")
print(f"📊 Plots directory: {plots_dir.absolute()}")

## 🔧 Data Loading and Processing Functions

In [ ]:
def safe_parse_prediction(x, debug_mode=False):
    """
    Safely parse predictions from string to list
    Handles various formats including dicts and strings
    """
    if pd.isna(x) or x == "[]" or x == "" or not isinstance(x, str):
        return []
    
    try:
        parsed = ast.literal_eval(x)
        
        if isinstance(parsed, list):
            # Extract strings from list (handles both str and dict)
            result = []
            for item in parsed:
                if isinstance(item, str):
                    result.append(item.strip())
                elif isinstance(item, dict):
                    # Try to extract a key field
                    if 'name' in item:
                        result.append(str(item['name']).strip())
                    elif 'poi' in item:
                        result.append(str(item['poi']).strip())
                    else:
                        # Take first dict value
                        values = list(item.values())
                        if values:
                            result.append(str(values[0]).strip())
                else:
                    result.append(str(item).strip())
            return result
        else:
            return []
    except Exception as e:
        if debug_mode:
            print(f"Parse error: {x} -> {e}")
        return []

def normalize_poi_name(poi_name):
    """
    Normalize POI name for consistent comparisons
    """
    if pd.isna(poi_name) or not isinstance(poi_name, str):
        return str(poi_name).strip()
    return poi_name.strip()

def calculate_metrics(df):
    """
    Calculate performance metrics for a dataset
    """
    def safe_top1_accuracy(row):
        pred_norm = row['prediction_norm']
        if not pred_norm or len(pred_norm) == 0:
            return False
        return pred_norm[0] == row['ground_truth_norm']
    
    def safe_top_k_hit(row, k=5):
        pred_norm = row['prediction_norm']
        if not pred_norm or len(pred_norm) == 0:
            return False
        return row['ground_truth_norm'] in pred_norm[:k]
    
    def safe_reciprocal_rank(row, k=5):
        pred_norm = row['prediction_norm']
        if not pred_norm or len(pred_norm) == 0:
            return 0.0
        try:
            rank = pred_norm[:k].index(row['ground_truth_norm']) + 1
            return 1.0 / rank
        except ValueError:
            return 0.0
    
    # Calculate metrics
    df['hit@1'] = df.apply(safe_top1_accuracy, axis=1)
    df['hit@5'] = df.apply(safe_top_k_hit, axis=1)
    df['rr'] = df.apply(safe_reciprocal_rank, axis=1)
    
    return df

print("✅ Data processing functions defined")

In [ ]:
def load_model_strategy_data(results_base_path='../results'):
    """
    Load data for all models and strategies from results directory
    Returns nested dictionary: {dataset_type: {model: {strategy: dataframe}}}
    """
    results_path = Path(results_base_path)
    
    # Available dataset types (excluding DEV)
    dataset_types = ['middle', 'penultimate']
    
    # Strategy types
    strategies = ['base_version', 'with_geom', 'with_geom_time']
    
    all_data = {}
    
    for dataset_type in dataset_types:
        dataset_path = results_path / dataset_type
        if not dataset_path.exists():
            print(f"⚠️ Dataset {dataset_type} not found, skipping")
            continue
            
        print(f"\n🔄 Loading {dataset_type} dataset...")
        all_data[dataset_type] = {}
        
        # Find all model directories
        model_dirs = [d for d in dataset_path.iterdir() if d.is_dir()]
        
        for model_dir in sorted(model_dirs):
            model_name = model_dir.name
            print(f"  📂 Processing model: {model_name}")
            all_data[dataset_type][model_name] = {}
            
            for strategy in strategies:
                strategy_path = model_dir / strategy
                if not strategy_path.exists():
                    print(f"    ⚠️ Strategy {strategy} not found for {model_name}")
                    continue
                    
                # Find CSV files in strategy directory
                csv_files = list(strategy_path.glob('*_pred_*.csv'))
                
                if not csv_files:
                    print(f"    ⚠️ No prediction CSV files found in {strategy}")
                    continue
                    
                print(f"    📊 Loading {strategy}: {len(csv_files)} files")
                
                # Load and concatenate all CSV files for this strategy
                strategy_dfs = []
                total_rows = 0
                
                for csv_file in sorted(csv_files):
                    try:
                        df = pd.read_csv(csv_file)
                        
                        # Extract year from filename
                        year_parts = [part for part in csv_file.stem.split('_') 
                                    if part.isdigit() and len(part) == 4]
                        if year_parts:
                            df['year'] = int(year_parts[0])
                        else:
                            print(f"      ⚠️ No year found in {csv_file.name}")
                            continue
                        
                        # Process predictions
                        df['prediction_list'] = df['prediction'].apply(safe_parse_prediction)
                        df['prediction_norm'] = df['prediction_list'].apply(
                            lambda x: [normalize_poi_name(poi) for poi in x] if isinstance(x, list) else []
                        )
                        df['ground_truth_norm'] = df['ground_truth'].apply(normalize_poi_name)
                        
                        # Filter valid predictions
                        df = df[df['prediction_list'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
                        
                        if len(df) > 0:
                            strategy_dfs.append(df)
                            total_rows += len(df)
                        
                    except Exception as e:
                        print(f"      ❌ Error loading {csv_file.name}: {e}")
                        continue
                
                if strategy_dfs:
                    # Concatenate all dataframes for this strategy
                    strategy_df = pd.concat(strategy_dfs, ignore_index=True)
                    
                    # Add metadata
                    strategy_df['model'] = model_name
                    strategy_df['strategy'] = strategy
                    strategy_df['dataset'] = dataset_type
                    
                    # Calculate metrics
                    strategy_df = calculate_metrics(strategy_df)
                    
                    all_data[dataset_type][model_name][strategy] = strategy_df
                    print(f"      ✅ {strategy}: {len(strategy_df):,} valid predictions")
                else:
                    print(f"      ❌ {strategy}: No valid data loaded")
    
    return all_data

# Load all data
print("🚀 Starting comprehensive data loading...")
all_model_data = load_model_strategy_data()

print("\n📊 LOADING SUMMARY:")
print("=" * 40)
for dataset_type, models in all_model_data.items():
    print(f"📁 {dataset_type.upper()}: {len(models)} models")
    for model_name, strategies in models.items():
        print(f"  🤖 {model_name}: {len(strategies)} strategies")
        for strategy_name, df in strategies.items():
            print(f"    📊 {strategy_name}: {len(df):,} predictions")

## 📊 Comprehensive Performance Analysis

In [ ]:
def create_comprehensive_comparison_df(all_data):
    """
    Create a comprehensive comparison DataFrame with all model-strategy combinations
    """
    comparison_data = []
    
    for dataset_type, models in all_data.items():
        for model_name, strategies in models.items():
            for strategy_name, df in strategies.items():
                if len(df) == 0:
                    continue
                
                # Calculate aggregate metrics
                metrics = {
                    'Dataset': dataset_type,
                    'Model': model_name,
                    'Strategy': strategy_name,
                    'Model_Display': model_name.replace('_', ' ').replace('-', ' ').title(),
                    'Strategy_Display': strategy_name.replace('_', ' ').title(),
                    'Combination': f"{model_name}_{strategy_name}",
                    'Total_Predictions': len(df),
                    'Top1_Accuracy': df['hit@1'].mean(),
                    'Top5_Hit_Rate': df['hit@5'].mean(),
                    'MRR': df['rr'].mean(),
                    'Years_Covered': len(df['year'].unique()),
                    'Year_Range': f"{df['year'].min()}-{df['year'].max()}",
                    'Unique_Users': df['card_id'].nunique() if 'card_id' in df.columns else len(df),
                }
                
                # Convert to percentages
                metrics['Top1_Accuracy_Percent'] = round(metrics['Top1_Accuracy'] * 100, 2)
                metrics['Top5_Hit_Rate_Percent'] = round(metrics['Top5_Hit_Rate'] * 100, 2)
                metrics['MRR_Percent'] = round(metrics['MRR'] * 100, 2)
                metrics['Error_Rate_Percent'] = round(100 - metrics['Top1_Accuracy_Percent'], 2)
                
                # Calculate standard deviations for confidence intervals
                metrics['Top1_Std'] = df['hit@1'].std()
                metrics['Top5_Std'] = df['hit@5'].std()
                metrics['MRR_Std'] = df['rr'].std()
                
                # Strategy encoding for analysis
                strategy_map = {
                    'base_version': 'Base',
                    'with_geom': 'Geospatial',
                    'with_geom_time': 'Geo+Temporal'
                }
                metrics['Strategy_Category'] = strategy_map.get(strategy_name, strategy_name)
                
                # Model size estimation (for analysis)
                if '7b' in model_name.lower():
                    metrics['Model_Size'] = '7B'
                elif '8b' in model_name.lower():
                    metrics['Model_Size'] = '8B'
                elif '14b' in model_name.lower():
                    metrics['Model_Size'] = '14B'
                elif '33b' in model_name.lower():
                    metrics['Model_Size'] = '33B'
                elif '8x7b' in model_name.lower():
                    metrics['Model_Size'] = '8x7B'
                else:
                    metrics['Model_Size'] = 'Unknown'
                
                comparison_data.append(metrics)
    
    return pd.DataFrame(comparison_data)

# Create comprehensive comparison DataFrame
comparison_df = create_comprehensive_comparison_df(all_model_data)

print(f"📊 Created comprehensive comparison with {len(comparison_df)} model-strategy combinations")
print(f"🤖 Models: {comparison_df['Model'].nunique()}")
print(f"🎯 Strategies: {comparison_df['Strategy'].nunique()}")
print(f"📁 Datasets: {comparison_df['Dataset'].nunique()}")

# Display top performers
print("\n🏆 TOP 10 PERFORMERS (by Top-1 Accuracy):")
top_performers = comparison_df.nlargest(10, 'Top1_Accuracy_Percent')
display(top_performers[['Model_Display', 'Strategy_Display', 'Dataset', 'Top1_Accuracy_Percent', 'Top5_Hit_Rate_Percent', 'Total_Predictions']])

## 📈 Visualization Generation

In [ ]:
# 1. Model Performance Comparison (grouped by strategy)
plt.figure(figsize=(15, 10))

# Filter to middle dataset for cleaner visualization
middle_df = comparison_df[comparison_df['Dataset'] == 'middle']

if len(middle_df) > 0:
    # Create pivot table for heatmap
    pivot_data = middle_df.pivot_table(
        index='Model_Display', 
        columns='Strategy_Category', 
        values='Top1_Accuracy_Percent',
        fill_value=0
    )
    
    # Create heatmap
    plt.subplot(2, 2, 1)
    sns.heatmap(pivot_data, annot=True, cmap='RdYlGn', fmt='.2f', 
                cbar_kws={'label': 'Top-1 Accuracy (%)'}, square=True)
    plt.title('Model Performance by Strategy\n(Top-1 Accuracy %)', fontsize=14, fontweight='bold')
    plt.xlabel('Strategy', fontweight='bold')
    plt.ylabel('Model', fontweight='bold')
    plt.xticks(rotation=45)
    
    # Bar plot comparing strategies
    plt.subplot(2, 2, 2)
    strategy_means = middle_df.groupby('Strategy_Category')['Top1_Accuracy_Percent'].mean().sort_values(ascending=False)
    bars = plt.bar(strategy_means.index, strategy_means.values, 
                   color=['#ff7f0e', '#2ca02c', '#d62728'][:len(strategy_means)])
    plt.title('Average Strategy Performance\n(Top-1 Accuracy)', fontsize=14, fontweight='bold')
    plt.ylabel('Top-1 Accuracy (%)', fontweight='bold')
    plt.xlabel('Strategy', fontweight='bold')
    
    # Add value labels on bars
    for bar, value in zip(bars, strategy_means.values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                f'{value:.2f}%', ha='center', fontweight='bold')
    
    # Model comparison (best strategy per model)
    plt.subplot(2, 2, 3)
    best_per_model = middle_df.loc[middle_df.groupby('Model')['Top1_Accuracy_Percent'].idxmax()]
    best_per_model = best_per_model.sort_values('Top1_Accuracy_Percent', ascending=True)
    
    bars = plt.barh(best_per_model['Model_Display'], best_per_model['Top1_Accuracy_Percent'])
    plt.title('Best Model Performance\n(Best Strategy per Model)', fontsize=14, fontweight='bold')
    plt.xlabel('Top-1 Accuracy (%)', fontweight='bold')
    
    # Color bars by strategy
    strategy_colors = {'Base': '#ff7f0e', 'Geospatial': '#2ca02c', 'Geo+Temporal': '#d62728'}
    for i, (_, row) in enumerate(best_per_model.iterrows()):
        bars[i].set_color(strategy_colors.get(row['Strategy_Category'], '#1f77b4'))
    
    # Add value labels
    for i, (_, row) in enumerate(best_per_model.iterrows()):
        plt.text(row['Top1_Accuracy_Percent'] + 0.1, i, 
                f"{row['Top1_Accuracy_Percent']:.2f}%\n({row['Strategy_Category']})", 
                va='center', fontsize=9)
    
    # Strategy improvement analysis
    plt.subplot(2, 2, 4)
    
    # Calculate improvement from base to advanced strategies
    improvement_data = []
    for model in middle_df['Model'].unique():
        model_data = middle_df[middle_df['Model'] == model]
        base_acc = model_data[model_data['Strategy'] == 'base_version']['Top1_Accuracy_Percent']
        geom_acc = model_data[model_data['Strategy'] == 'with_geom']['Top1_Accuracy_Percent']
        geom_time_acc = model_data[model_data['Strategy'] == 'with_geom_time']['Top1_Accuracy_Percent']
        
        if len(base_acc) > 0:
            base_val = base_acc.iloc[0]
            if len(geom_acc) > 0:
                improvement_data.append({
                    'Model': model.replace('_', ' ').title(),
                    'Strategy': 'Geospatial vs Base',
                    'Improvement': geom_acc.iloc[0] - base_val
                })
            if len(geom_time_acc) > 0:
                improvement_data.append({
                    'Model': model.replace('_', ' ').title(),
                    'Strategy': 'Geo+Temporal vs Base',
                    'Improvement': geom_time_acc.iloc[0] - base_val
                })
    
    if improvement_data:
        imp_df = pd.DataFrame(improvement_data)
        
        # Create grouped bar chart
        models = imp_df['Model'].unique()
        x = np.arange(len(models))
        width = 0.35
        
        geom_improvements = []
        geom_time_improvements = []
        
        for model in models:
            geom_imp = imp_df[(imp_df['Model'] == model) & (imp_df['Strategy'] == 'Geospatial vs Base')]['Improvement']
            geom_time_imp = imp_df[(imp_df['Model'] == model) & (imp_df['Strategy'] == 'Geo+Temporal vs Base')]['Improvement']
            
            geom_improvements.append(geom_imp.iloc[0] if len(geom_imp) > 0 else 0)
            geom_time_improvements.append(geom_time_imp.iloc[0] if len(geom_time_imp) > 0 else 0)
        
        plt.bar(x - width/2, geom_improvements, width, label='Geospatial vs Base', color='#2ca02c', alpha=0.8)
        plt.bar(x + width/2, geom_time_improvements, width, label='Geo+Temporal vs Base', color='#d62728', alpha=0.8)
        
        plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        plt.title('Strategy Improvement Analysis\n(Percentage Points vs Base)', fontsize=14, fontweight='bold')
        plt.xlabel('Model', fontweight='bold')
        plt.ylabel('Accuracy Improvement (pp)', fontweight='bold')
        plt.xticks(x, [m.replace(' ', '\n') for m in models], rotation=0, fontsize=9)
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save plot
    plot_path = plots_dir / 'comprehensive_model_strategy_comparison.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"📊 Saved comprehensive comparison plot: {plot_path}")
    
    plt.show()
else:
    print("⚠️ No middle dataset found for visualization")

In [ ]:
# 2. Detailed Performance Distribution Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

if len(middle_df) > 0:
    # Box plot: Performance distribution by strategy
    sns.boxplot(data=middle_df, x='Strategy_Category', y='Top1_Accuracy_Percent', ax=axes[0,0])
    axes[0,0].set_title('Performance Distribution by Strategy', fontsize=14, fontweight='bold')
    axes[0,0].set_ylabel('Top-1 Accuracy (%)', fontweight='bold')
    axes[0,0].set_xlabel('Strategy', fontweight='bold')
    
    # Violin plot: Performance distribution by model size
    sns.violinplot(data=middle_df, x='Model_Size', y='Top1_Accuracy_Percent', ax=axes[0,1])
    axes[0,1].set_title('Performance Distribution by Model Size', fontsize=14, fontweight='bold')
    axes[0,1].set_ylabel('Top-1 Accuracy (%)', fontweight='bold')
    axes[0,1].set_xlabel('Model Size', fontweight='bold')
    
    # Scatter plot: Model Size vs Performance
    # Convert model size to numeric for scatter plot
    size_mapping = {'7B': 7, '8B': 8, '14B': 14, '33B': 33, '8x7B': 56, 'Unknown': 0}
    middle_df_scatter = middle_df.copy()
    middle_df_scatter['Model_Size_Numeric'] = middle_df_scatter['Model_Size'].map(size_mapping)
    middle_df_scatter = middle_df_scatter[middle_df_scatter['Model_Size_Numeric'] > 0]
    
    scatter = axes[1,0].scatter(middle_df_scatter['Model_Size_Numeric'], 
                              middle_df_scatter['Top1_Accuracy_Percent'],
                              c=middle_df_scatter['Strategy_Category'].astype('category').cat.codes,
                              s=middle_df_scatter['Total_Predictions'] / 1000,  # Size by number of predictions
                              alpha=0.7, cmap='Set1')
    
    axes[1,0].set_xlabel('Model Size (Billions of Parameters)', fontweight='bold')
    axes[1,0].set_ylabel('Top-1 Accuracy (%)', fontweight='bold')
    axes[1,0].set_title('Model Size vs Performance\n(Bubble size = # Predictions)', fontsize=14, fontweight='bold')
    
    # Add trend line
    z = np.polyfit(middle_df_scatter['Model_Size_Numeric'], middle_df_scatter['Top1_Accuracy_Percent'], 1)
    p = np.poly1d(z)
    axes[1,0].plot(middle_df_scatter['Model_Size_Numeric'], p(middle_df_scatter['Model_Size_Numeric']), 
                   "r--", alpha=0.8, linewidth=2)
    
    # Performance vs Data Volume
    axes[1,1].scatter(middle_df['Total_Predictions'], middle_df['Top1_Accuracy_Percent'], 
                     c=middle_df['Strategy_Category'].astype('category').cat.codes,
                     alpha=0.7, s=60, cmap='Set1')
    axes[1,1].set_xlabel('Total Predictions', fontweight='bold')
    axes[1,1].set_ylabel('Top-1 Accuracy (%)', fontweight='bold')
    axes[1,1].set_title('Performance vs Dataset Size', fontsize=14, fontweight='bold')
    axes[1,1].set_xscale('log')
    
    # Add strategy legend
    strategy_categories = middle_df['Strategy_Category'].unique()
    colors = plt.cm.Set1(np.linspace(0, 1, len(strategy_categories)))
    legend_elements = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=colors[i], 
                                 markersize=10, label=cat) for i, cat in enumerate(strategy_categories)]
    axes[1,1].legend(handles=legend_elements, title='Strategy', loc='upper right')
    
    plt.tight_layout()
    
    # Save plot
    plot_path = plots_dir / 'detailed_performance_analysis.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"📊 Saved detailed performance analysis: {plot_path}")
    
    plt.show()
else:
    print("⚠️ No data available for detailed analysis")

## 📤 Export Canva-Ready CSV Files

In [ ]:
# Export comprehensive comparison data
print("🔄 Generating Canva-ready CSV files...\n")

# 1. Model-Strategy Comparison (Main comparison table)
model_strategy_csv = comparison_df[[
    'Dataset', 'Model_Display', 'Strategy_Display', 'Strategy_Category',
    'Top1_Accuracy_Percent', 'Top5_Hit_Rate_Percent', 'MRR_Percent', 
    'Error_Rate_Percent', 'Total_Predictions', 'Years_Covered',
    'Model_Size', 'Year_Range'
]].copy()

# Add rankings
model_strategy_csv['Rank_Overall'] = model_strategy_csv['Top1_Accuracy_Percent'].rank(ascending=False, method='min')
model_strategy_csv['Rank_Within_Strategy'] = model_strategy_csv.groupby('Strategy_Category')['Top1_Accuracy_Percent'].rank(ascending=False, method='min')
model_strategy_csv['Rank_Within_Model'] = model_strategy_csv.groupby('Model_Display')['Top1_Accuracy_Percent'].rank(ascending=False, method='min')

# Performance categories
model_strategy_csv['Performance_Tier'] = pd.cut(
    model_strategy_csv['Top1_Accuracy_Percent'],
    bins=[0, 2, 4, 6, 8, 100],
    labels=['Basic', 'Fair', 'Good', 'Very Good', 'Excellent']
)

output_file = output_dir / 'comprehensive_model_strategy_comparison.csv'
model_strategy_csv.to_csv(output_file, index=False)
print(f"✅ Exported comprehensive comparison: {output_file}")
print(f"   📊 {len(model_strategy_csv)} model-strategy combinations")

# 2. Strategy Effectiveness Summary
strategy_summary = comparison_df.groupby(['Strategy_Category', 'Dataset']).agg({
    'Top1_Accuracy_Percent': ['mean', 'std', 'count'],
    'Top5_Hit_Rate_Percent': ['mean', 'std'],
    'MRR_Percent': ['mean', 'std'],
    'Total_Predictions': 'sum'
}).round(2)

# Flatten column names
strategy_summary.columns = [
    'Top1_Mean', 'Top1_Std', 'Models_Count', 
    'Top5_Mean', 'Top5_Std',
    'MRR_Mean', 'MRR_Std', 
    'Total_Predictions'
]

strategy_summary = strategy_summary.reset_index()
strategy_summary['Error_Rate_Mean'] = 100 - strategy_summary['Top1_Mean']

output_file = output_dir / 'strategy_effectiveness_summary.csv'
strategy_summary.to_csv(output_file, index=False)
print(f"✅ Exported strategy summary: {output_file}")
print(f"   📊 {len(strategy_summary)} strategy-dataset combinations")

# 3. Model Performance Summary  
model_summary = comparison_df.groupby(['Model_Display', 'Model_Size', 'Dataset']).agg({
    'Top1_Accuracy_Percent': ['mean', 'std', 'max'],
    'Top5_Hit_Rate_Percent': ['mean', 'max'],
    'MRR_Percent': ['mean', 'max'],
    'Strategy_Category': lambda x: ', '.join(x.unique()),  # Available strategies
    'Total_Predictions': 'sum'
}).round(2)

# Flatten column names
model_summary.columns = [
    'Top1_Mean', 'Top1_Std', 'Top1_Best',
    'Top5_Mean', 'Top5_Best',
    'MRR_Mean', 'MRR_Best',
    'Available_Strategies', 'Total_Predictions'
]

model_summary = model_summary.reset_index()
model_summary['Strategy_Count'] = model_summary['Available_Strategies'].str.count(',') + 1

# Add best strategy for each model
best_strategies = comparison_df.loc[comparison_df.groupby(['Model_Display', 'Dataset'])['Top1_Accuracy_Percent'].idxmax()]
best_strategy_map = dict(zip(best_strategies['Model_Display'] + '_' + best_strategies['Dataset'], 
                            best_strategies['Strategy_Category']))
model_summary['Best_Strategy'] = (model_summary['Model_Display'] + '_' + model_summary['Dataset']).map(best_strategy_map)

output_file = output_dir / 'model_performance_summary.csv'
model_summary.to_csv(output_file, index=False)
print(f"✅ Exported model summary: {output_file}")
print(f"   📊 {len(model_summary)} model-dataset combinations")

# 4. Strategy Improvement Analysis
improvement_analysis = []

for dataset in comparison_df['Dataset'].unique():
    dataset_df = comparison_df[comparison_df['Dataset'] == dataset]
    
    for model in dataset_df['Model'].unique():
        model_data = dataset_df[dataset_df['Model'] == model]
        
        base_row = model_data[model_data['Strategy'] == 'base_version']
        geom_row = model_data[model_data['Strategy'] == 'with_geom']
        geom_time_row = model_data[model_data['Strategy'] == 'with_geom_time']
        
        if len(base_row) > 0:
            base_acc = base_row['Top1_Accuracy_Percent'].iloc[0]
            model_display = base_row['Model_Display'].iloc[0]
            model_size = base_row['Model_Size'].iloc[0]
            
            # Geospatial improvement
            if len(geom_row) > 0:
                geom_acc = geom_row['Top1_Accuracy_Percent'].iloc[0]
                improvement_analysis.append({
                    'Dataset': dataset,
                    'Model': model_display,
                    'Model_Size': model_size,
                    'Comparison': 'Geospatial vs Base',
                    'Base_Accuracy': base_acc,
                    'Enhanced_Accuracy': geom_acc,
                    'Improvement_Points': round(geom_acc - base_acc, 2),
                    'Improvement_Percent': round(((geom_acc - base_acc) / base_acc) * 100, 2)
                })
            
            # Temporal improvement
            if len(geom_time_row) > 0:
                geom_time_acc = geom_time_row['Top1_Accuracy_Percent'].iloc[0]
                improvement_analysis.append({
                    'Dataset': dataset,
                    'Model': model_display,
                    'Model_Size': model_size,
                    'Comparison': 'Geo+Temporal vs Base',
                    'Base_Accuracy': base_acc,
                    'Enhanced_Accuracy': geom_time_acc,
                    'Improvement_Points': round(geom_time_acc - base_acc, 2),
                    'Improvement_Percent': round(((geom_time_acc - base_acc) / base_acc) * 100, 2)
                })
            
            # Geom+Time vs Geom
            if len(geom_row) > 0 and len(geom_time_row) > 0:
                geom_acc = geom_row['Top1_Accuracy_Percent'].iloc[0]
                geom_time_acc = geom_time_row['Top1_Accuracy_Percent'].iloc[0]
                improvement_analysis.append({
                    'Dataset': dataset,
                    'Model': model_display,
                    'Model_Size': model_size,
                    'Comparison': 'Geo+Temporal vs Geospatial',
                    'Base_Accuracy': geom_acc,
                    'Enhanced_Accuracy': geom_time_acc,
                    'Improvement_Points': round(geom_time_acc - geom_acc, 2),
                    'Improvement_Percent': round(((geom_time_acc - geom_acc) / geom_acc) * 100, 2)
                })

improvement_df = pd.DataFrame(improvement_analysis)

if len(improvement_df) > 0:
    # Add improvement categories
    improvement_df['Improvement_Category'] = pd.cut(
        improvement_df['Improvement_Points'],
        bins=[-100, -1, -0.1, 0.1, 1, 100],
        labels=['Large Decrease', 'Small Decrease', 'No Change', 'Small Increase', 'Large Increase']
    )
    
    output_file = output_dir / 'strategy_improvement_analysis.csv'
    improvement_df.to_csv(output_file, index=False)
    print(f"✅ Exported improvement analysis: {output_file}")
    print(f"   📊 {len(improvement_df)} strategy comparisons")
else:
    print("⚠️ No improvement analysis data available")

# 5. Top Performers Summary (for presentation highlights)
top_performers_summary = []

# Overall top performer
overall_best = comparison_df.loc[comparison_df['Top1_Accuracy_Percent'].idxmax()]
top_performers_summary.append({
    'Category': 'Overall Best',
    'Model': overall_best['Model_Display'],
    'Strategy': overall_best['Strategy_Display'],
    'Dataset': overall_best['Dataset'],
    'Top1_Accuracy_Percent': overall_best['Top1_Accuracy_Percent'],
    'Total_Predictions': overall_best['Total_Predictions']
})

# Best per strategy
for strategy in comparison_df['Strategy_Category'].unique():
    strategy_best = comparison_df[comparison_df['Strategy_Category'] == strategy].loc[
        comparison_df[comparison_df['Strategy_Category'] == strategy]['Top1_Accuracy_Percent'].idxmax()
    ]
    
    top_performers_summary.append({
        'Category': f'Best {strategy}',
        'Model': strategy_best['Model_Display'],
        'Strategy': strategy_best['Strategy_Display'],
        'Dataset': strategy_best['Dataset'],
        'Top1_Accuracy_Percent': strategy_best['Top1_Accuracy_Percent'],
        'Total_Predictions': strategy_best['Total_Predictions']
    })

# Best per model size
for size in comparison_df['Model_Size'].unique():
    if size != 'Unknown':
        size_best = comparison_df[comparison_df['Model_Size'] == size].loc[
            comparison_df[comparison_df['Model_Size'] == size]['Top1_Accuracy_Percent'].idxmax()
        ]
        
        top_performers_summary.append({
            'Category': f'Best {size} Model',
            'Model': size_best['Model_Display'],
            'Strategy': size_best['Strategy_Display'],
            'Dataset': size_best['Dataset'],
            'Top1_Accuracy_Percent': size_best['Top1_Accuracy_Percent'],
            'Total_Predictions': size_best['Total_Predictions']
        })

top_performers_df = pd.DataFrame(top_performers_summary)

output_file = output_dir / 'top_performers_highlights.csv'
top_performers_df.to_csv(output_file, index=False)
print(f"✅ Exported top performers: {output_file}")
print(f"   🏆 {len(top_performers_df)} highlight categories")

print(f"\n🎯 ALL CSV FILES EXPORTED TO: {output_dir.absolute()}")
print(f"📊 Ready for import into Canva for professional visualizations!")

## 📊 Summary Statistics & Insights

In [ ]:
print("📈 COMPREHENSIVE ANALYSIS SUMMARY")
print("=" * 50)

# Overall statistics
total_combinations = len(comparison_df)
unique_models = comparison_df['Model'].nunique()
unique_strategies = comparison_df['Strategy'].nunique()
total_predictions = comparison_df['Total_Predictions'].sum()

print(f"🔍 Analyzed {total_combinations} model-strategy combinations")
print(f"🤖 {unique_models} unique models tested")
print(f"🎯 {unique_strategies} different strategies")
print(f"📊 {total_predictions:,} total predictions processed")

print(f"\n🏆 PERFORMANCE INSIGHTS:")
print("-" * 30)

# Best overall performance
best_overall = comparison_df.loc[comparison_df['Top1_Accuracy_Percent'].idxmax()]
print(f"🥇 Best Overall: {best_overall['Model_Display']} with {best_overall['Strategy_Display']}")
print(f"   📊 Top-1 Accuracy: {best_overall['Top1_Accuracy_Percent']:.2f}%")
print(f"   📈 Top-5 Hit Rate: {best_overall['Top5_Hit_Rate_Percent']:.2f}%")

# Strategy effectiveness
print(f"\n🎯 STRATEGY EFFECTIVENESS:")
strategy_performance = comparison_df.groupby('Strategy_Category')['Top1_Accuracy_Percent'].agg(['mean', 'std', 'count'])
strategy_performance = strategy_performance.sort_values('mean', ascending=False)

for strategy, row in strategy_performance.iterrows():
    print(f"   {strategy}: {row['mean']:.2f}% ± {row['std']:.2f}% ({int(row['count'])} models)")

# Model size analysis
print(f"\n💾 MODEL SIZE ANALYSIS:")
size_performance = comparison_df[comparison_df['Model_Size'] != 'Unknown'].groupby('Model_Size')['Top1_Accuracy_Percent'].agg(['mean', 'std', 'count'])
size_performance = size_performance.sort_values('mean', ascending=False)

for size, row in size_performance.iterrows():
    print(f"   {size}: {row['mean']:.2f}% ± {row['std']:.2f}% ({int(row['count'])} combinations)")

# Improvement analysis
if len(improvement_df) > 0:
    print(f"\n📈 STRATEGY IMPROVEMENTS:")
    print("-" * 25)
    
    improvement_summary = improvement_df.groupby('Comparison')['Improvement_Points'].agg(['mean', 'std', 'count'])
    improvement_summary = improvement_summary.sort_values('mean', ascending=False)
    
    for comparison, row in improvement_summary.iterrows():
        direction = "📈" if row['mean'] > 0 else "📉" if row['mean'] < 0 else "➡️"
        print(f"   {direction} {comparison}: {row['mean']:+.2f} pp ± {row['std']:.2f} ({int(row['count'])} models)")
    
    # Best improvements
    best_improvements = improvement_df.nlargest(3, 'Improvement_Points')
    print(f"\n🚀 TOP 3 IMPROVEMENTS:")
    for _, row in best_improvements.iterrows():
        print(f"   {row['Model']} ({row['Comparison']}): +{row['Improvement_Points']:.2f} pp")

print(f"\n💡 KEY INSIGHTS:")
print("-" * 15)

# Generate automatic insights
insights = []

# Strategy insight
best_strategy = strategy_performance.index[0]
worst_strategy = strategy_performance.index[-1]
strategy_gap = strategy_performance.loc[best_strategy, 'mean'] - strategy_performance.loc[worst_strategy, 'mean']
insights.append(f"🎯 {best_strategy} strategy outperforms {worst_strategy} by {strategy_gap:.1f} percentage points")

# Model size insight
if len(size_performance) > 1:
    largest_model = size_performance.index[0]
    insights.append(f"💾 {largest_model} models show the best average performance ({size_performance.loc[largest_model, 'mean']:.1f}%)")

# Consistency insight
most_consistent_strategy = strategy_performance.loc[strategy_performance['std'].idxmin()].name
insights.append(f"🎯 {most_consistent_strategy} strategy shows most consistent results across models")

# Improvement insight
if len(improvement_df) > 0:
    avg_geom_improvement = improvement_df[improvement_df['Comparison'] == 'Geospatial vs Base']['Improvement_Points'].mean()
    avg_temporal_improvement = improvement_df[improvement_df['Comparison'] == 'Geo+Temporal vs Base']['Improvement_Points'].mean()
    
    if avg_geom_improvement > 0:
        insights.append(f"📈 Adding geospatial features improves performance by {avg_geom_improvement:.1f} pp on average")
    if avg_temporal_improvement > 0:
        insights.append(f"⏰ Full geo+temporal enhancement provides {avg_temporal_improvement:.1f} pp average improvement")

for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

print(f"\n✅ Analysis complete! All data exported and ready for presentation.")
print(f"📁 Find your files in: {output_dir.absolute()}")
print(f"📊 Find your plots in: {plots_dir.absolute()}")

In [ ]:
# Final verification and file listing
print("📋 FINAL FILE VERIFICATION")
print("=" * 30)

# List all generated files
csv_files = list(output_dir.glob('*.csv'))
plot_files = list(plots_dir.glob('*.png'))

print(f"📊 CSV FILES ({len(csv_files)}):")
for csv_file in sorted(csv_files):
    size = csv_file.stat().st_size
    try:
        df_check = pd.read_csv(csv_file)
        print(f"  ✅ {csv_file.name} ({df_check.shape[0]}×{df_check.shape[1]}, {size:,} bytes)")
    except Exception as e:
        print(f"  ❌ {csv_file.name} (Error: {e})")

print(f"\n🖼️ PLOT FILES ({len(plot_files)}):")
for plot_file in sorted(plot_files):
    size = plot_file.stat().st_size
    print(f"  ✅ {plot_file.name} ({size:,} bytes)")

total_size = sum(f.stat().st_size for f in csv_files + plot_files)
print(f"\n💾 Total output size: {total_size:,} bytes ({total_size/1024/1024:.1f} MB)")

print(f"\n🎨 CANVA IMPORT TIPS:")
print("- Use comprehensive_model_strategy_comparison.csv for main comparison charts")
print("- Use strategy_effectiveness_summary.csv for strategy-focused visualizations")
print("- Use top_performers_highlights.csv for presentation highlights")
print("- All percentages are pre-formatted (e.g., 4.32 not 0.0432)")
print("- Performance tiers are already categorized for easy color coding")
print("- Rankings are provided for easy sorting and highlighting")
print("")
print("🚀 Ready for professional presentation creation!")

In [ ]:
# ADVANCED VISUALIZATION 1: Radar Chart for Model Comparison
import math

def create_radar_chart_data():
    """
    Create radar chart data for multi-dimensional model comparison
    """
    try:
        # Calculate multiple performance dimensions for each model
        radar_data = []
        
        for model in comparison_df['Model'].unique():
            model_data = comparison_df[comparison_df['Model'] == model]
            
            # Calculate average performance across strategies
            avg_top1 = model_data['Top1_Accuracy_Percent'].mean()
            avg_top5 = model_data['Top5_Hit_Rate_Percent'].mean()
            avg_mrr = model_data['MRR_Percent'].mean()
            
            # Calculate consistency (inverse of standard deviation)
            consistency = 100 - (model_data['Top1_Accuracy_Percent'].std() * 10)  # Scale for radar
            consistency = max(0, min(100, consistency))  # Clamp between 0-100
            
            # Calculate data coverage (based on total predictions)
            max_predictions = comparison_df['Total_Predictions'].max()
            coverage = (model_data['Total_Predictions'].sum() / max_predictions) * 100
            coverage = min(100, coverage)  # Cap at 100%
            
            # Calculate strategy versatility (number of strategies available)
            strategy_count = len(model_data)
            versatility = (strategy_count / 3) * 100  # Assuming max 3 strategies
            
            radar_data.append({
                'Model': model.replace('_', ' ').title(),
                'Model_Code': model,
                'Top1_Accuracy': avg_top1,
                'Top5_Hit_Rate': avg_top5,
                'MRR': avg_mrr,
                'Consistency': consistency,
                'Data_Coverage': coverage,
                'Strategy_Versatility': versatility
            })
        
        return pd.DataFrame(radar_data)
    
    except Exception as e:
        print(f"❌ Error creating radar chart data: {e}")
        return pd.DataFrame()

# Create and export radar chart data
radar_df = create_radar_chart_data()

if not radar_df.empty:
    # Export for Canva
    output_file = output_dir / 'model_radar_comparison.csv'
    radar_df.to_csv(output_file, index=False)
    print(f"✅ Exported radar chart data: {output_file}")
    print(f"   📊 {len(radar_df)} models with 6 performance dimensions")
    
    # Create visualization
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))
    
    # Categories for radar chart
    categories = ['Top1_Accuracy', 'Top5_Hit_Rate', 'MRR', 'Consistency', 'Data_Coverage', 'Strategy_Versatility']
    category_labels = ['Top-1\nAccuracy', 'Top-5\nHit Rate', 'Mean\nReciprocal\nRank', 'Consistency', 'Data\nCoverage', 'Strategy\nVersatility']
    
    # Number of variables
    N = len(categories)
    
    # Compute angles for each category
    angles = [n / float(N) * 2 * math.pi for n in range(N)]
    angles += angles[:1]  # Complete the circle
    
    # Plot data for each model
    colors = plt.cm.Set3(np.linspace(0, 1, len(radar_df)))
    
    for idx, (_, row) in enumerate(radar_df.iterrows()):
        values = [row[cat] for cat in categories]
        values += values[:1]  # Complete the circle
        
        ax.plot(angles, values, 'o-', linewidth=2, label=row['Model'], color=colors[idx], alpha=0.8)
        ax.fill(angles, values, alpha=0.1, color=colors[idx])
    
    # Customize the chart
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(category_labels, fontsize=10)
    ax.set_ylim(0, 100)
    ax.set_yticks([20, 40, 60, 80, 100])
    ax.set_yticklabels(['20%', '40%', '60%', '80%', '100%'], fontsize=8)
    ax.grid(True, alpha=0.3)
    
    # Add legend
    ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.1), fontsize=9)
    
    plt.title('Multi-Dimensional Model Performance Comparison\n(Radar Chart)', 
              fontsize=14, fontweight='bold', y=1.08)
    
    # Save plot
    plot_path = plots_dir / 'model_radar_comparison.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"📊 Saved radar chart: {plot_path}")
    
    plt.tight_layout()
    plt.show()
    
    # Display the data table
    print("\n📋 RADAR CHART DATA:")
    display(radar_df.round(2))
else:
    print("⚠️ Could not create radar chart data")

In [ ]:
# ADVANCED VISUALIZATION 2: Performance Evolution & Temporal Analysis
def create_temporal_analysis():
    """
    Analyze performance trends across years and create temporal visualizations
    """
    try:
        temporal_data = []
        
        # Collect all individual prediction data
        all_predictions = []
        for dataset_type, models in all_model_data.items():
            for model_name, strategies in models.items():
                for strategy_name, df in strategies.items():
                    if len(df) > 0:
                        sample_df = df.copy()
                        sample_df['model'] = model_name
                        sample_df['strategy'] = strategy_name
                        sample_df['dataset'] = dataset_type
                        all_predictions.append(sample_df)
        
        if all_predictions:
            combined_predictions = pd.concat(all_predictions, ignore_index=True)
            
            # Calculate yearly performance for each model-strategy combination
            yearly_performance = combined_predictions.groupby(['model', 'strategy', 'dataset', 'year']).agg({
                'hit@1': ['mean', 'count'],
                'hit@5': 'mean',
                'rr': 'mean'
            }).round(4)
            
            # Flatten column names
            yearly_performance.columns = ['Top1_Accuracy', 'Predictions_Count', 'Top5_Hit_Rate', 'MRR']
            yearly_performance = yearly_performance.reset_index()
            
            # Convert to percentages
            for col in ['Top1_Accuracy', 'Top5_Hit_Rate', 'MRR']:
                yearly_performance[f'{col}_Percent'] = (yearly_performance[col] * 100).round(2)
            
            # Add display names
            yearly_performance['Model_Display'] = yearly_performance['model'].str.replace('_', ' ').str.replace('-', ' ').str.title()
            yearly_performance['Strategy_Display'] = yearly_performance['strategy'].str.replace('_', ' ').str.title()
            
            return yearly_performance
        else:
            print("⚠️ No temporal data available")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"❌ Error in temporal analysis: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

# Create temporal analysis
temporal_df = create_temporal_analysis()

if not temporal_df.empty:
    # Export temporal data
    output_file = output_dir / 'temporal_performance_analysis.csv'
    temporal_df.to_csv(output_file, index=False)
    print(f"✅ Exported temporal analysis: {output_file}")
    print(f"   📊 {len(temporal_df)} year-model-strategy combinations")
    
    # Create temporal visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Filter for better visualization (focus on models with multiple years)
    models_with_data = temporal_df.groupby(['model', 'strategy']).size()
    multi_year_combinations = models_with_data[models_with_data > 3].index.tolist()
    
    if multi_year_combinations:
        # Plot 1: Performance trends over years for top models
        top_combinations = []
        for model, strategy in multi_year_combinations[:4]:  # Top 4 combinations
            combo_data = temporal_df[(temporal_df['model'] == model) & (temporal_df['strategy'] == strategy)]
            if len(combo_data) > 0:
                top_combinations.append((model, strategy, combo_data))
        
        for idx, (model, strategy, data) in enumerate(top_combinations):
            label = f"{model.replace('_', ' ').title()} ({strategy.replace('_', ' ').title()})"
            axes[0,0].plot(data['year'], data['Top1_Accuracy_Percent'], 
                          marker='o', label=label, linewidth=2, alpha=0.8)
        
        axes[0,0].set_title('Performance Trends Over Years\n(Top Model-Strategy Combinations)', 
                           fontsize=14, fontweight='bold')
        axes[0,0].set_xlabel('Year', fontweight='bold')
        axes[0,0].set_ylabel('Top-1 Accuracy (%)', fontweight='bold')
        axes[0,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
        axes[0,0].grid(True, alpha=0.3)
    
    # Plot 2: Year-over-year performance distribution
    if 'year' in temporal_df.columns and len(temporal_df['year'].unique()) > 1:
        sns.boxplot(data=temporal_df, x='year', y='Top1_Accuracy_Percent', ax=axes[0,1])
        axes[0,1].set_title('Performance Distribution by Year', fontsize=14, fontweight='bold')
        axes[0,1].set_xlabel('Year', fontweight='bold')
        axes[0,1].set_ylabel('Top-1 Accuracy (%)', fontweight='bold')
        axes[0,1].tick_params(axis='x', rotation=45)
    
    # Plot 3: Dataset volume vs performance over years
    if len(temporal_df) > 10:
        scatter = axes[1,0].scatter(temporal_df['Predictions_Count'], 
                                  temporal_df['Top1_Accuracy_Percent'],
                                  c=temporal_df['year'], cmap='viridis', 
                                  alpha=0.6, s=50)
        axes[1,0].set_xlabel('Predictions Count (per year)', fontweight='bold')
        axes[1,0].set_ylabel('Top-1 Accuracy (%)', fontweight='bold')
        axes[1,0].set_title('Dataset Volume vs Performance\n(Color = Year)', fontsize=14, fontweight='bold')
        axes[1,0].set_xscale('log')
        plt.colorbar(scatter, ax=axes[1,0], label='Year')
    
    # Plot 4: Performance consistency across years
    if len(temporal_df) > 5:
        # Calculate coefficient of variation for each model-strategy
        consistency_data = []
        for (model, strategy), group in temporal_df.groupby(['model', 'strategy']):
            if len(group) > 2:  # Need at least 3 years for meaningful CV
                cv = (group['Top1_Accuracy_Percent'].std() / group['Top1_Accuracy_Percent'].mean()) * 100
                consistency_data.append({
                    'Model_Strategy': f"{model.replace('_', ' ')[:10]}\n({strategy.replace('_', ' ')[:8]})",
                    'Consistency_Score': max(0, 100 - cv),  # Higher score = more consistent
                    'Mean_Performance': group['Top1_Accuracy_Percent'].mean(),
                    'Years_Count': len(group)
                })
        
        if consistency_data:
            consistency_df = pd.DataFrame(consistency_data)
            consistency_df = consistency_df.sort_values('Mean_Performance', ascending=True).tail(8)  # Top 8
            
            bars = axes[1,1].barh(consistency_df['Model_Strategy'], consistency_df['Consistency_Score'])
            axes[1,1].set_xlabel('Consistency Score (0-100)', fontweight='bold')
            axes[1,1].set_title('Performance Consistency Across Years\n(Higher = More Stable)', 
                               fontsize=14, fontweight='bold')
            
            # Color bars by mean performance
            norm = plt.Normalize(consistency_df['Mean_Performance'].min(), consistency_df['Mean_Performance'].max())
            for bar, perf in zip(bars, consistency_df['Mean_Performance']):
                bar.set_color(plt.cm.RdYlGn(norm(perf)))
    
    plt.tight_layout()
    
    # Save plot
    plot_path = plots_dir / 'temporal_performance_analysis.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"📊 Saved temporal analysis plot: {plot_path}")
    
    plt.show()
    
    # Show summary statistics
    print("\n📈 TEMPORAL PERFORMANCE SUMMARY:")
    print("=" * 35)
    
    if len(temporal_df['year'].unique()) > 1:
        yearly_avg = temporal_df.groupby('year')['Top1_Accuracy_Percent'].mean().sort_index()
        print("📅 Average performance by year:")
        for year, perf in yearly_avg.items():
            print(f"   {year}: {perf:.2f}%")
        
        # Calculate trend
        years = yearly_avg.index.values
        perfs = yearly_avg.values
        if len(years) > 1:
            trend = np.polyfit(years, perfs, 1)[0]  # Linear trend coefficient
            direction = "📈 Improving" if trend > 0 else "📉 Declining" if trend < 0 else "➡️ Stable"
            print(f"\n🎯 Overall trend: {direction} ({trend:+.3f}% per year)")
    
else:
    print("⚠️ Could not create temporal analysis")

In [ ]:
# ENHANCED ERROR HANDLING & DIAGNOSTIC FUNCTIONS

def enhanced_error_handling():
    """
    Comprehensive error handling and diagnostic system
    """
    
    class DataProcessingError(Exception):
        """Custom exception for data processing errors"""
        pass
    
    class FileValidationError(Exception):
        """Custom exception for file validation errors"""
        pass
    
    def validate_csv_file(filepath, required_columns=None, min_rows=1):
        """
        Validate CSV file structure and content
        
        Args:
            filepath (Path): Path to CSV file
            required_columns (list): List of required column names
            min_rows (int): Minimum number of rows required
            
        Returns:
            dict: Validation result with status and details
        """
        try:
            if not filepath.exists():
                return {
                    'status': 'error',
                    'message': f'File does not exist: {filepath}',
                    'suggestions': ['Check file path', 'Ensure file was created successfully']
                }
            
            # Try to read the file
            try:
                df = pd.read_csv(filepath)
            except pd.errors.EmptyDataError:
                return {
                    'status': 'error',
                    'message': 'File is empty',
                    'suggestions': ['Check data generation process', 'Verify data sources']
                }
            except pd.errors.ParserError as e:
                return {
                    'status': 'error',
                    'message': f'CSV parsing error: {e}',
                    'suggestions': [
                        'Check CSV formatting',
                        'Look for inconsistent delimiters',
                        'Check for embedded quotes or newlines'
                    ]
                }
            
            # Validate row count
            if len(df) < min_rows:
                return {
                    'status': 'warning',
                    'message': f'File has only {len(df)} rows (minimum: {min_rows})',
                    'suggestions': ['Check data filtering logic', 'Verify data source completeness']
                }
            
            # Validate required columns
            if required_columns:
                missing_cols = [col for col in required_columns if col not in df.columns]
                if missing_cols:
                    return {
                        'status': 'error',
                        'message': f'Missing required columns: {missing_cols}',
                        'available_columns': list(df.columns),
                        'suggestions': [
                            'Check column name spelling',
                            'Verify data processing pipeline',
                            'Check for column name changes'
                        ]
                    }
            
            # Check for data quality issues
            quality_issues = []
            
            # Check for missing values
            missing_counts = df.isnull().sum()
            critical_missing = missing_counts[missing_counts > len(df) * 0.5]
            if len(critical_missing) > 0:
                quality_issues.append(f'High missing values in: {list(critical_missing.index)}')
            
            # Check for duplicate rows
            if df.duplicated().sum() > 0:
                quality_issues.append(f'{df.duplicated().sum()} duplicate rows found')
            
            # Check for numeric columns with invalid values
            numeric_cols = df.select_dtypes(include=[np.number]).columns
            for col in numeric_cols:
                if df[col].isna().sum() == len(df):
                    quality_issues.append(f'Column {col} is entirely empty')
                elif (df[col] < 0).any() and 'Percent' in col:
                    quality_issues.append(f'Negative percentages in {col}')
                elif (df[col] > 100).any() and 'Percent' in col:
                    quality_issues.append(f'Percentages > 100% in {col}')
            
            return {
                'status': 'success' if not quality_issues else 'warning',
                'message': f'File validated successfully ({len(df)} rows, {len(df.columns)} columns)',
                'quality_issues': quality_issues,
                'file_info': {
                    'rows': len(df),
                    'columns': len(df.columns),
                    'size_mb': filepath.stat().st_size / 1024 / 1024,
                    'column_names': list(df.columns)
                }
            }
            
        except Exception as e:
            return {
                'status': 'error',
                'message': f'Unexpected validation error: {str(e)}',
                'suggestions': [
                    'Check file permissions',
                    'Verify file is not corrupted',
                    'Try opening file manually'
                ]
            }
    
    def diagnose_data_loading_issues(all_model_data):
        """
        Diagnose common data loading issues and provide solutions
        """
        print("🔍 DATA LOADING DIAGNOSTIC REPORT")
        print("=" * 40)
        
        total_models = 0
        total_strategies = 0
        total_predictions = 0
        issues_found = []
        
        for dataset_type, models in all_model_data.items():
            print(f"\n📁 Dataset: {dataset_type.upper()}")
            
            if not models:
                issues_found.append(f"No models found in {dataset_type} dataset")
                print("   ❌ No models loaded")
                continue
                
            dataset_predictions = 0
            
            for model_name, strategies in models.items():
                total_models += 1
                model_predictions = 0
                
                print(f"   🤖 {model_name}:")
                
                if not strategies:
                    issues_found.append(f"No strategies found for {model_name} in {dataset_type}")
                    print("      ❌ No strategies loaded")
                    continue
                
                for strategy_name, df in strategies.items():
                    total_strategies += 1
                    strategy_predictions = len(df) if df is not None else 0
                    model_predictions += strategy_predictions
                    
                    if strategy_predictions == 0:
                        issues_found.append(f"Empty dataset: {model_name}/{strategy_name} in {dataset_type}")
                        print(f"      ⚠️ {strategy_name}: No data")
                    elif strategy_predictions < 1000:
                        issues_found.append(f"Small dataset: {model_name}/{strategy_name} has only {strategy_predictions} predictions")
                        print(f"      ⚠️ {strategy_name}: {strategy_predictions:,} predictions (small)")
                    else:
                        print(f"      ✅ {strategy_name}: {strategy_predictions:,} predictions")
                
                dataset_predictions += model_predictions
                if model_predictions == 0:
                    issues_found.append(f"Model {model_name} has no valid data in {dataset_type}")
            
            total_predictions += dataset_predictions
            print(f"   📊 Dataset total: {dataset_predictions:,} predictions")
        
        print(f"\n📈 OVERALL SUMMARY:")
        print(f"   🤖 Total models: {total_models}")
        print(f"   🎯 Total strategies: {total_strategies}")
        print(f"   📊 Total predictions: {total_predictions:,}")
        
        if issues_found:
            print(f"\n⚠️ ISSUES IDENTIFIED ({len(issues_found)}):")
            for i, issue in enumerate(issues_found, 1):
                print(f"   {i}. {issue}")
            
            print(f"\n🔧 TROUBLESHOOTING SUGGESTIONS:")
            print("   • Check that CSV files exist in the results directories")
            print("   • Verify CSV files are not corrupted (try opening manually)")
            print("   • Check file permissions and paths")
            print("   • Ensure prediction columns contain valid data")
            print("   • Look for missing or incomplete model runs")
            print("   • Check for consistent CSV formatting across files")
        else:
            print("\n✅ No major issues detected!")
        
        return {
            'total_models': total_models,
            'total_strategies': total_strategies,
            'total_predictions': total_predictions,
            'issues': issues_found
        }
    
    return validate_csv_file, diagnose_data_loading_issues, DataProcessingError, FileValidationError

# Initialize error handling functions
validate_csv_file, diagnose_data_loading_issues, DataProcessingError, FileValidationError = enhanced_error_handling()

# Run diagnostic on loaded data
diagnostic_result = diagnose_data_loading_issues(all_model_data)

print(f"\n🔍 VALIDATION OF EXPORTED CSV FILES:")
print("=" * 40)

# Validate all exported CSV files
csv_files = list(output_dir.glob('*.csv'))
validation_results = {}

for csv_file in csv_files:
    print(f"\n📄 Validating {csv_file.name}...")
    
    # Define expected columns for each file type
    expected_columns = {
        'comprehensive_model_strategy_comparison.csv': [
            'Model_Display', 'Strategy_Display', 'Top1_Accuracy_Percent', 'Dataset'
        ],
        'strategy_effectiveness_summary.csv': [
            'Strategy_Category', 'Dataset', 'Top1_Mean', 'Models_Count'
        ],
        'model_performance_summary.csv': [
            'Model_Display', 'Model_Size', 'Top1_Best', 'Best_Strategy'
        ],
        'top_performers_highlights.csv': [
            'Category', 'Model', 'Strategy', 'Top1_Accuracy_Percent'
        ]
    }
    
    required_cols = expected_columns.get(csv_file.name, [])
    result = validate_csv_file(csv_file, required_cols, min_rows=1)
    validation_results[csv_file.name] = result
    
    if result['status'] == 'success':
        print(f"   ✅ {result['message']}")
        if 'file_info' in result:
            info = result['file_info']
            print(f"      📊 {info['rows']} rows × {info['columns']} columns ({info['size_mb']:.2f} MB)")
    elif result['status'] == 'warning':
        print(f"   ⚠️ {result['message']}")
        if 'quality_issues' in result and result['quality_issues']:
            for issue in result['quality_issues']:
                print(f"      • {issue}")
    else:
        print(f"   ❌ {result['message']}")
        if 'suggestions' in result:
            print("      Suggestions:")
            for suggestion in result['suggestions']:
                print(f"        • {suggestion}")

print(f"\n📊 VALIDATION SUMMARY:")
success_count = sum(1 for r in validation_results.values() if r['status'] == 'success')
warning_count = sum(1 for r in validation_results.values() if r['status'] == 'warning')
error_count = sum(1 for r in validation_results.values() if r['status'] == 'error')

print(f"   ✅ Successful: {success_count}/{len(validation_results)}")
print(f"   ⚠️ Warnings: {warning_count}/{len(validation_results)}")
print(f"   ❌ Errors: {error_count}/{len(validation_results)}")

if error_count == 0:
    print(f"\n🎉 All CSV files are valid and ready for Canva import!")
else:
    print(f"\n⚠️ Some files have issues that need attention before use.")

In [ ]:
# COMPREHENSIVE CSV DOCUMENTATION & ERROR HANDLING

def create_csv_documentation():
    """
    Create comprehensive documentation for all exported CSV files
    with detailed explanations, use cases, and data schemas
    """
    csv_docs = {
        'comprehensive_model_strategy_comparison.csv': {
            'title': '🎯 Comprehensive Model-Strategy Comparison',
            'description': 'Main comparison dataset with all model-strategy combinations and performance metrics',
            'use_cases': [
                'Main comparison charts in Canva (bar charts, heatmaps)',
                'Ranking visualizations and performance tables',
                'Model selection decision support',
                'Strategy effectiveness comparison'
            ],
            'key_columns': {
                'Dataset': 'middle or penultimate - data source',
                'Model_Display': 'Human-readable model names (e.g., "Qwen2.5 14B")',
                'Strategy_Display': 'Human-readable strategy names (e.g., "With Geom")',
                'Top1_Accuracy_Percent': 'Top-1 accuracy as percentage (0-100)',
                'Top5_Hit_Rate_Percent': 'Top-5 hit rate as percentage (0-100)',
                'MRR_Percent': 'Mean Reciprocal Rank as percentage (0-100)',
                'Error_Rate_Percent': '100 - Top1_Accuracy_Percent',
                'Total_Predictions': 'Number of predictions made',
                'Rank_Overall': 'Global ranking by Top-1 accuracy (1=best)',
                'Performance_Tier': 'Categorical tier (Basic/Fair/Good/Very Good/Excellent)',
                'Model_Size': 'Parameter count category (7B, 8B, 14B, 33B, 8x7B)'
            },
            'canva_tips': [
                'Use Rank_Overall for sorting charts',
                'Color-code by Performance_Tier',
                'Filter by Dataset for cleaner comparisons',
                'Use Model_Size for grouping visualizations'
            ]
        },
        
        'strategy_effectiveness_summary.csv': {
            'title': '🎯 Strategy Effectiveness Summary',
            'description': 'Aggregated performance statistics for each strategy across all models',
            'use_cases': [
                'Strategy comparison bar charts',
                'Statistical confidence intervals',
                'Strategy recommendation support',
                'Performance distribution analysis'
            ],
            'key_columns': {
                'Strategy_Category': 'Base, Geospatial, or Geo+Temporal',
                'Dataset': 'middle or penultimate',
                'Top1_Mean': 'Average Top-1 accuracy across models (percentage)',
                'Top1_Std': 'Standard deviation of Top-1 accuracy',
                'Models_Count': 'Number of models tested with this strategy',
                'Error_Rate_Mean': '100 - Top1_Mean'
            },
            'canva_tips': [
                'Use Top1_Mean for main bar heights',
                'Add error bars using Top1_Std',
                'Show Models_Count as data labels',
                'Use Error_Rate_Mean for failure analysis'
            ]
        },
        
        'model_performance_summary.csv': {
            'title': '🤖 Model Performance Summary',
            'description': 'Comprehensive performance statistics for each model across all strategies',
            'use_cases': [
                'Model comparison charts',
                'Best strategy identification per model',
                'Model capability assessment',
                'Resource allocation decisions'
            ],
            'key_columns': {
                'Model_Display': 'Human-readable model name',
                'Model_Size': 'Parameter count category',
                'Top1_Best': 'Best Top-1 accuracy achieved by this model',
                'Available_Strategies': 'Comma-separated list of tested strategies',
                'Best_Strategy': 'Strategy that achieved Top1_Best performance',
                'Strategy_Count': 'Number of strategies tested for this model'
            },
            'canva_tips': [
                'Sort by Top1_Best for ranking charts',
                'Use Model_Size for grouping',
                'Highlight Best_Strategy in annotations',
                'Size bubbles by Strategy_Count'
            ]
        },
        
        'strategy_improvement_analysis.csv': {
            'title': '📈 Strategy Improvement Analysis',
            'description': 'Detailed analysis of performance improvements when upgrading strategies',
            'use_cases': [
                'Improvement waterfall charts',
                'Strategy ROI visualization',
                'Decision support for strategy upgrades',
                'Cost-benefit presentations'
            ],
            'key_columns': {
                'Comparison': 'Type of comparison (e.g., "Geospatial vs Base")',
                'Base_Accuracy': 'Performance of baseline strategy',
                'Enhanced_Accuracy': 'Performance of enhanced strategy',
                'Improvement_Points': 'Absolute improvement in percentage points',
                'Improvement_Percent': 'Relative improvement as percentage',
                'Improvement_Category': 'Categorical improvement level'
            },
            'canva_tips': [
                'Use Improvement_Points for bar charts',
                'Color by Improvement_Category',
                'Show both baseline and enhanced values',
                'Filter positive improvements for success stories'
            ]
        },
        
        'top_performers_highlights.csv': {
            'title': '🏆 Top Performers Highlights',
            'description': 'Key highlights and top performers for presentation summaries',
            'use_cases': [
                'Executive summary slides',
                'Key findings highlights',
                'Award-style presentations',
                'Quick fact sheets'
            ],
            'key_columns': {
                'Category': 'Type of achievement (e.g., "Overall Best", "Best 14B Model")',
                'Model': 'Winning model name',
                'Strategy': 'Winning strategy name',
                'Top1_Accuracy_Percent': 'Achievement value'
            },
            'canva_tips': [
                'Create award cards for each category',
                'Use as callout boxes in presentations',
                'Perfect for infographic highlights',
                'Great for social media snippets'
            ]
        },
        
        'model_radar_comparison.csv': {
            'title': '📡 Multi-Dimensional Model Radar Data',
            'description': 'Multi-dimensional performance data optimized for radar/spider charts',
            'use_cases': [
                'Radar/spider charts showing model strengths',
                'Multi-dimensional comparisons',
                'Holistic model assessment',
                'Complex performance visualization'
            ],
            'key_columns': {
                'Model': 'Model display name',
                'Top1_Accuracy': 'Average accuracy performance (0-100)',
                'Top5_Hit_Rate': 'Average hit rate performance (0-100)',
                'Consistency': 'Performance consistency score (0-100)',
                'Data_Coverage': 'Data volume coverage score (0-100)',
                'Strategy_Versatility': 'Strategy availability score (0-100)'
            },
            'canva_tips': [
                'Perfect for radar charts in Canva',
                'All values scaled 0-100 for consistency',
                'Compare models across multiple dimensions',
                'Great for capability assessment'
            ]
        },
        
        'temporal_performance_analysis.csv': {
            'title': '⏰ Temporal Performance Analysis',
            'description': 'Year-by-year performance data for trend analysis',
            'use_cases': [
                'Time series line charts',
                'Performance trend analysis',
                'Yearly comparison tables',
                'Temporal stability assessment'
            ],
            'key_columns': {
                'model': 'Model code name',
                'strategy': 'Strategy code name',
                'year': 'Data year (2014-2023)',
                'Top1_Accuracy_Percent': 'Yearly Top-1 accuracy',
                'Predictions_Count': 'Number of predictions that year'
            },
            'canva_tips': [
                'Use year as X-axis for line charts',
                'Filter by model/strategy for focused trends',
                'Size points by Predictions_Count',
                'Show confidence through data volume'
            ]
        },
        
        'cost_benefit_analysis.csv': {
            'title': '💰 Cost-Benefit Analysis',
            'description': 'Comprehensive cost-benefit analysis with ROI calculations',
            'use_cases': [
                'ROI comparison charts',
                'Cost vs performance scatter plots',
                'Budget allocation decisions',
                'Resource optimization visualization'
            ],
            'key_columns': {
                'Computational_Cost': 'Estimated relative computational cost',
                'Performance': 'Top-1 accuracy performance',
                'ROI_Score': 'Return on investment score',
                'Efficiency_Score': 'Performance per cost unit',
                'Cost_Category': 'Low/Medium/High cost classification',
                'Benefit_Category': 'Low/Medium/High benefit classification'
            },
            'canva_tips': [
                'Use ROI_Score for ranking charts',
                'Create cost vs performance scatter plots',
                'Color by Cost_Category or Benefit_Category',
                'Size bubbles by Efficiency_Score'
            ]
        }
    }
    
    return csv_docs

# Create documentation
csv_documentation = create_csv_documentation()

print("📚 COMPREHENSIVE CSV DOCUMENTATION")
print("=" * 50)
print("\n🎯 This documentation covers all exported CSV files with detailed")
print("   explanations, use cases, and Canva integration tips.\n")

for filename, doc in csv_documentation.items():
    print(f"📄 {doc['title']}")
    print(f"   File: {filename}")
    print(f"   {doc['description']}\n")
    
    print("   📊 USE CASES:")
    for use_case in doc['use_cases']:
        print(f"      • {use_case}")
    
    if 'key_columns' in doc:
        print("\n   🏷️ KEY COLUMNS:")
        for col, desc in doc['key_columns'].items():
            print(f"      • {col}: {desc}")
    
    print("\n   🎨 CANVA TIPS:")
    for tip in doc['canva_tips']:
        print(f"      • {tip}")
    
    print("\n" + "-" * 60 + "\n")

In [ ]:
# ADVANCED VISUALIZATION 2: Performance Evolution & Temporal Analysis
def create_temporal_analysis():
    """
    Analyze performance trends across years and create temporal visualizations
    """
    try:
        temporal_data = []
        
        # Collect all individual prediction data
        all_predictions = []
        for dataset_type, models in all_model_data.items():
            for model_name, strategies in models.items():
                for strategy_name, df in strategies.items():
                    if len(df) > 0:
                        sample_df = df.copy()
                        sample_df['model'] = model_name
                        sample_df['strategy'] = strategy_name
                        sample_df['dataset'] = dataset_type
                        all_predictions.append(sample_df)
        
        if all_predictions:
            combined_predictions = pd.concat(all_predictions, ignore_index=True)
            
            # Calculate yearly performance for each model-strategy combination
            yearly_performance = combined_predictions.groupby(['model', 'strategy', 'dataset', 'year']).agg({
                'hit@1': ['mean', 'count'],
                'hit@5': 'mean',
                'rr': 'mean'
            }).round(4)
            
            # Flatten column names
            yearly_performance.columns = ['Top1_Accuracy', 'Predictions_Count', 'Top5_Hit_Rate', 'MRR']
            yearly_performance = yearly_performance.reset_index()
            
            # Convert to percentages
            for col in ['Top1_Accuracy', 'Top5_Hit_Rate', 'MRR']:
                yearly_performance[f'{col}_Percent'] = (yearly_performance[col] * 100).round(2)
            
            # Add display names
            yearly_performance['Model_Display'] = yearly_performance['model'].str.replace('_', ' ').str.replace('-', ' ').str.title()
            yearly_performance['Strategy_Display'] = yearly_performance['strategy'].str.replace('_', ' ').str.title()
            
            return yearly_performance
        else:
            print("⚠️ No temporal data available")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"❌ Error in temporal analysis: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

# Create temporal analysis
temporal_df = create_temporal_analysis()

if not temporal_df.empty:
    # Export temporal data
    output_file = output_dir / 'temporal_performance_analysis.csv'
    temporal_df.to_csv(output_file, index=False)
    print(f"✅ Exported temporal analysis: {output_file}")
    print(f"   📊 {len(temporal_df)} year-model-strategy combinations")
    
    # Create temporal visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Filter for better visualization (focus on models with multiple years)
    models_with_data = temporal_df.groupby(['model', 'strategy']).size()
    multi_year_combinations = models_with_data[models_with_data > 3].index.tolist()
    
    if multi_year_combinations:
        # Plot 1: Performance trends over years for top models
        top_combinations = []
        for model, strategy in multi_year_combinations[:4]:  # Top 4 combinations
            combo_data = temporal_df[(temporal_df['model'] == model) & (temporal_df['strategy'] == strategy)]
            if len(combo_data) > 0:
                top_combinations.append((model, strategy, combo_data))
        
        for idx, (model, strategy, data) in enumerate(top_combinations):
            label = f"{model.replace('_', ' ').title()} ({strategy.replace('_', ' ').title()})"
            axes[0,0].plot(data['year'], data['Top1_Accuracy_Percent'], 
                          marker='o', label=label, linewidth=2, alpha=0.8)
        
        axes[0,0].set_title('Performance Trends Over Years\\n(Top Model-Strategy Combinations)', 
                           fontsize=14, fontweight='bold')
        axes[0,0].set_xlabel('Year', fontweight='bold')
        axes[0,0].set_ylabel('Top-1 Accuracy (%)', fontweight='bold')
        axes[0,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
        axes[0,0].grid(True, alpha=0.3)
    
    # Plot 2: Year-over-year performance distribution
    if 'year' in temporal_df.columns and len(temporal_df['year'].unique()) > 1:
        sns.boxplot(data=temporal_df, x='year', y='Top1_Accuracy_Percent', ax=axes[0,1])
        axes[0,1].set_title('Performance Distribution by Year', fontsize=14, fontweight='bold')
        axes[0,1].set_xlabel('Year', fontweight='bold')
        axes[0,1].set_ylabel('Top-1 Accuracy (%)', fontweight='bold')
        axes[0,1].tick_params(axis='x', rotation=45)
    
    # Plot 3: Dataset volume vs performance over years
    if len(temporal_df) > 10:
        scatter = axes[1,0].scatter(temporal_df['Predictions_Count'], 
                                  temporal_df['Top1_Accuracy_Percent'],
                                  c=temporal_df['year'], cmap='viridis', 
                                  alpha=0.6, s=50)
        axes[1,0].set_xlabel('Predictions Count (per year)', fontweight='bold')
        axes[1,0].set_ylabel('Top-1 Accuracy (%)', fontweight='bold')
        axes[1,0].set_title('Dataset Volume vs Performance\\n(Color = Year)', fontsize=14, fontweight='bold')
        axes[1,0].set_xscale('log')
        plt.colorbar(scatter, ax=axes[1,0], label='Year')
    
    # Plot 4: Performance consistency across years
    if len(temporal_df) > 5:
        # Calculate coefficient of variation for each model-strategy
        consistency_data = []
        for (model, strategy), group in temporal_df.groupby(['model', 'strategy']):
            if len(group) > 2:  # Need at least 3 years for meaningful CV
                cv = (group['Top1_Accuracy_Percent'].std() / group['Top1_Accuracy_Percent'].mean()) * 100
                consistency_data.append({
                    'Model_Strategy': f"{model.replace('_', ' ')[:10]}\\n({strategy.replace('_', ' ')[:8]})",
                    'Consistency_Score': max(0, 100 - cv),  # Higher score = more consistent
                    'Mean_Performance': group['Top1_Accuracy_Percent'].mean(),
                    'Years_Count': len(group)
                })
        
        if consistency_data:
            consistency_df = pd.DataFrame(consistency_data)
            consistency_df = consistency_df.sort_values('Mean_Performance', ascending=True).tail(8)  # Top 8
            
            bars = axes[1,1].barh(consistency_df['Model_Strategy'], consistency_df['Consistency_Score'])
            axes[1,1].set_xlabel('Consistency Score (0-100)', fontweight='bold')
            axes[1,1].set_title('Performance Consistency Across Years\\n(Higher = More Stable)', 
                               fontsize=14, fontweight='bold')
            
            # Color bars by mean performance
            norm = plt.Normalize(consistency_df['Mean_Performance'].min(), consistency_df['Mean_Performance'].max())
            for bar, perf in zip(bars, consistency_df['Mean_Performance']):
                bar.set_color(plt.cm.RdYlGn(norm(perf)))
    
    plt.tight_layout()
    
    # Save plot
    plot_path = plots_dir / 'temporal_performance_analysis.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"📊 Saved temporal analysis plot: {plot_path}")
    
    plt.show()
    
    # Show summary statistics
    print("\\n📈 TEMPORAL PERFORMANCE SUMMARY:")
    print("=" * 35)
    
    if len(temporal_df['year'].unique()) > 1:
        yearly_avg = temporal_df.groupby('year')['Top1_Accuracy_Percent'].mean().sort_index()
        print("📅 Average performance by year:")
        for year, perf in yearly_avg.items():
            print(f"   {year}: {perf:.2f}%")
        
        # Calculate trend
        years = yearly_avg.index.values
        perfs = yearly_avg.values
        if len(years) > 1:
            trend = np.polyfit(years, perfs, 1)[0]  # Linear trend coefficient
            direction = "📈 Improving" if trend > 0 else "📉 Declining" if trend < 0 else "➡️ Stable"
            print(f"\\n🎯 Overall trend: {direction} ({trend:+.3f}% per year)")
    
else:
    print("⚠️ Could not create temporal analysis")

In [ ]:
# ADVANCED VISUALIZATION 1: Radar Chart for Model Comparison
import math

def create_radar_chart_data():
    """
    Create radar chart data for multi-dimensional model comparison
    """
    try:
        # Calculate multiple performance dimensions for each model
        radar_data = []
        
        for model in comparison_df['Model'].unique():
            model_data = comparison_df[comparison_df['Model'] == model]
            
            # Calculate average performance across strategies
            avg_top1 = model_data['Top1_Accuracy_Percent'].mean()
            avg_top5 = model_data['Top5_Hit_Rate_Percent'].mean()
            avg_mrr = model_data['MRR_Percent'].mean()
            
            # Calculate consistency (inverse of standard deviation)
            consistency = 100 - (model_data['Top1_Accuracy_Percent'].std() * 10)  # Scale for radar
            consistency = max(0, min(100, consistency))  # Clamp between 0-100
            
            # Calculate data coverage (based on total predictions)
            max_predictions = comparison_df['Total_Predictions'].max()
            coverage = (model_data['Total_Predictions'].sum() / max_predictions) * 100
            coverage = min(100, coverage)  # Cap at 100%
            
            # Calculate strategy versatility (number of strategies available)
            strategy_count = len(model_data)
            versatility = (strategy_count / 3) * 100  # Assuming max 3 strategies
            
            radar_data.append({
                'Model': model.replace('_', ' ').title(),
                'Model_Code': model,
                'Top1_Accuracy': avg_top1,
                'Top5_Hit_Rate': avg_top5,
                'MRR': avg_mrr,
                'Consistency': consistency,
                'Data_Coverage': coverage,
                'Strategy_Versatility': versatility
            })
        
        return pd.DataFrame(radar_data)
    
    except Exception as e:
        print(f"❌ Error creating radar chart data: {e}")
        return pd.DataFrame()

# Create and export radar chart data
radar_df = create_radar_chart_data()

if not radar_df.empty:
    # Export for Canva
    output_file = output_dir / 'model_radar_comparison.csv'
    radar_df.to_csv(output_file, index=False)
    print(f"✅ Exported radar chart data: {output_file}")
    print(f"   📊 {len(radar_df)} models with 6 performance dimensions")
    
    # Create visualization
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))
    
    # Categories for radar chart
    categories = ['Top1_Accuracy', 'Top5_Hit_Rate', 'MRR', 'Consistency', 'Data_Coverage', 'Strategy_Versatility']
    category_labels = ['Top-1\nAccuracy', 'Top-5\nHit Rate', 'Mean\nReciprocal\nRank', 'Consistency', 'Data\nCoverage', 'Strategy\nVersatility']
    
    # Number of variables
    N = len(categories)
    
    # Compute angles for each category
    angles = [n / float(N) * 2 * math.pi for n in range(N)]
    angles += angles[:1]  # Complete the circle
    
    # Plot data for each model
    colors = plt.cm.Set3(np.linspace(0, 1, len(radar_df)))
    
    for idx, (_, row) in enumerate(radar_df.iterrows()):
        values = [row[cat] for cat in categories]
        values += values[:1]  # Complete the circle
        
        ax.plot(angles, values, 'o-', linewidth=2, label=row['Model'], color=colors[idx], alpha=0.8)
        ax.fill(angles, values, alpha=0.1, color=colors[idx])
    
    # Customize the chart
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(category_labels, fontsize=10)
    ax.set_ylim(0, 100)
    ax.set_yticks([20, 40, 60, 80, 100])
    ax.set_yticklabels(['20%', '40%', '60%', '80%', '100%'], fontsize=8)
    ax.grid(True, alpha=0.3)
    
    # Add legend
    ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.1), fontsize=9)
    
    plt.title('Multi-Dimensional Model Performance Comparison\\n(Radar Chart)', 
              fontsize=14, fontweight='bold', y=1.08)
    
    # Save plot
    plot_path = plots_dir / 'model_radar_comparison.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"📊 Saved radar chart: {plot_path}")
    
    plt.tight_layout()
    plt.show()
    
    # Display the data table
    print("\\n📋 RADAR CHART DATA:")
    display(radar_df.round(2))
else:
    print("⚠️ Could not create radar chart data")

## 🔥 Additional Advanced Visualizations & Analysis

This section contains advanced visualizations and analytics specifically designed for professional presentations.